In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import time
from utils import *
from models import *
from attention import *

import torch
import torch.optim as optim
from torch.utils import data
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
# Download the file
get_file()
path_to_file = 'spa-eng/spa.txt'

======== THE DATASET IS READY ========


In [4]:
# Try experimenting with the size of that dataset
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [5]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

24000 24000 6000 6000


In [6]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
4280 ----> sirvanse
154 ----> otra
2531 ----> copa
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
29 ----> have
372 ----> another
748 ----> cup
3 ----> .
2 ----> <end>


In [7]:
input_tensor_train

array([[   1, 4280,  154, ...,    0,    0,    0],
       [   1,    9,   17, ...,    0,    0,    0],
       [   1,    6,   52, ...,    0,    0,    0],
       ...,
       [   1, 3832,   31, ...,    0,    0,    0],
       [   1,   38,  642, ...,    0,    0,    0],
       [   1,    7,   61, ...,    0,    0,    0]], dtype=int32)

In [8]:
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

In [9]:
# transform to torch tensor
tensor_x = torch.Tensor(input_tensor_train).long() 
tensor_y = torch.Tensor(target_tensor_train).long()
# create your datset
my_dataset = data.TensorDataset(tensor_x,tensor_y) 
# create your dataloader
my_dataloader = data.DataLoader(my_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        num_workers=4)   

In [10]:
my_dataset[20]

(tensor([  1, 501,  10,   4,   3,   2,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0]),
 tensor([ 1, 28, 43,  5,  3,  2,  0,  0,  0,  0,  0]))

In [11]:
example_input_batch, example_target_batch = next(iter(my_dataloader))

In [12]:
example_input_batch.shape

torch.Size([64, 16])

In [13]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) torch.Size([64, 16, 1024])
Encoder Hidden state shape: (batch size, units) torch.Size([1, 64, 1024])


In [14]:
attention_layer = BahdanauAttention(10, 1024)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) torch.Size([64, 1024])
Attention weights shape: (batch_size, sequence_length, 1) torch.Size([64, 16, 1])


In [15]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 1024)

sample_decoder_output, _, _ = decoder(torch.randint(1, 20, (BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) torch.Size([64, 4935])


In [16]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE).to(device)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 1024).to(device)

encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [17]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    #enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(my_dataloader):
        inp, targ = inp.to(device), targ.to(device)
        batch_loss = train_step(inp, targ, encoder, decoder,
                                encoder_optimizer, decoder_optimizer,
                                criterion, device, BATCH_SIZE, targ_lang)
        
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss))
            
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        
        pass

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 7.7095
Epoch 1 Batch 100 Loss 1.7956
Epoch 1 Batch 200 Loss 1.4834
Epoch 1 Batch 300 Loss 1.1799
Epoch 1 Loss 1.5976
Time taken for 1 epoch 40.82125377655029 sec

Epoch 2 Batch 0 Loss 0.7981
Epoch 2 Batch 100 Loss 0.7514
Epoch 2 Batch 200 Loss 0.7692
Epoch 2 Batch 300 Loss 0.7020
Epoch 2 Loss 0.7684
Time taken for 1 epoch 37.96637535095215 sec

Epoch 3 Batch 0 Loss 0.3821
Epoch 3 Batch 100 Loss 0.3398
Epoch 3 Batch 200 Loss 0.3608
Epoch 3 Batch 300 Loss 0.4561
Epoch 3 Loss 0.4173
Time taken for 1 epoch 37.8361234664917 sec

Epoch 4 Batch 0 Loss 0.2914
Epoch 4 Batch 100 Loss 0.2483
Epoch 4 Batch 200 Loss 0.2497
Epoch 4 Batch 300 Loss 0.2356
Epoch 4 Loss 0.2374
Time taken for 1 epoch 39.842841148376465 sec

Epoch 5 Batch 0 Loss 0.1293
Epoch 5 Batch 100 Loss 0.1336
Epoch 5 Batch 200 Loss 0.1632
Epoch 5 Batch 300 Loss 0.1716
Epoch 5 Loss 0.1502
Time taken for 1 epoch 37.5951452255249 sec

Epoch 6 Batch 0 Loss 0.0819
Epoch 6 Batch 100 Loss 0.0986
Epoch 6 Batch 200 Loss 